In [1]:
!pip install langchain-groq langchain faiss-cpu langchain_community sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 975.5/975.5 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 49.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.5/103.5 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 332.8/332.8 kB 20.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.4/127.4 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.0/145.0 kB 2.4 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using c

In [2]:
import json
import pickle
import pandas as pd
import numpy as np
from getpass import getpass
from tqdm import tqdm
from typing import List
import natsort
import faiss
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.docstore import InMemoryDocstore
from langchain.schema import Document
from langchain.vectorstores import FAISS
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor
from langchain_groq import ChatGroq
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field

In [3]:
GROQ_API_KEY = getpass("Enter your Groq API key: ")

Enter your Groq API key: ··········


In [4]:
llama3_70b_temp0 = ChatGroq(
    temperature=0,
    model_name="llama3-70b-8192",
    api_key=GROQ_API_KEY)

# Test
llama3_70b_temp0.invoke("What is the capital of France?")

AIMessage(content='The capital of France is Paris.', response_metadata={'token_usage': {'completion_tokens': 8, 'prompt_tokens': 17, 'total_tokens': 25, 'completion_time': 0.022857143, 'prompt_time': 0.004933946, 'queue_time': None, 'total_time': 0.027791088999999998}, 'model_name': 'llama3-70b-8192', 'system_fingerprint': 'fp_7ab5f7e105', 'finish_reason': 'stop', 'logprobs': None}, id='run-f30899c9-0ae8-4866-b71c-8132d0d5267d-0')

In [5]:
df = pd.read_csv("/content/dataset_concurso_rag.csv")
df.columns, len(df)

(Index(['pergunta', 'resposta', 'comentarios', 'numero_na_captura', 'id',
        'area', 'ano', 'banca', 'prova', 'duvidou_do_gabarito', 'enunciado',
        'assertivas', 'alternativas', 'classificacao',
        'artigos_nos_comentarios', 'artigos_completos', 'artigo_inventado',
        'topicos_para_busca', 'artigos_candidatos', 'frase_assunto_da_questao',
        'rag_art_inventado_full', 'rag_art_inventado_full_sem_art',
        'rag_assunto_full', 'rag_topicos_full', 'rag_artigos_candidatos',
        'rag_artigos_candidatos_grupos'],
       dtype='object'),
 37)

In [6]:
df[['rag_art_inventado_full', 'rag_art_inventado_full_sem_art', 'rag_assunto_full', 'rag_topicos_full', 'rag_artigos_candidatos', 'rag_artigos_candidatos_grupos']]

,rag_art_inventado_full,rag_art_inventado_full_sem_art,rag_assunto_full,rag_topicos_full,rag_artigos_candidatos,rag_artigos_candidatos_grupos
0,"Art. 1.238. Aquele que, por quinze anos, sem i...","Art. 1.238. Aquele que, por quinze anos, sem i...","Art. 1.824. O herdeiro pode, em ação de petiçã...",Art. 1.210. O possuidor tem direito a ser mant...,Art. 1.815. A exclusão do herdeiro ou legatári...,Art. 1.814. São excluídos da sucessão os herde...
1,Art. 1.976. O testador pode nomear um ou mais ...,Art. 1.976. O testador pode nomear um ou mais ...,Art. 26. Decorrido um ano da arrecadação dos b...,"Art. 1.784. Aberta a sucessão, a herança trans...",Art. 1.862. São testamentos ordinários:I - o p...,Art. 1.862. São testamentos ordinários:\nI - o...
2,Art. 1.876. O testamento particular pode ser e...,Art. 1.876. O testamento particular pode ser e...,"Art. 1.857. Toda pessoa capaz pode dispor, por...",Art. 212. Salvo o negócio a que se impõe forma...,"Art. 1.857. Toda pessoa capaz pode dispor, por...",Art. 1.862. São testamentos ordinários:\nI - o...
3,Art. 1.976. O testador pode nomear um ou mais ...,Art. 1.976. O testador pode nomear um ou mais ...,Art. 1.976. O testador pode nomear um ou mais ...,Art. 92. Principal é o bem que existe sobre si...,"Art. 1.848. Salvo se houver justa causa, decla...","Art. 1.860. Além dos incapazes, não podem test..."
4,Art. 1.723. É reconhecida como entidade famili...,Art. 1.723. É reconhecida como entidade famili...,Art. 1.723. É reconhecida como entidade famili...,Art. 1.723. É reconhecida como entidade famili...,Art. 1.723. É reconhecida como entidade famili...,"Art. 1.784. Aberta a sucessão, a herança trans..."
5,"Art. 1.022. A sociedade adquire direitos, assu...","Art. 1.022. A sociedade adquire direitos, assu...","Art. 1.022. A sociedade adquire direitos, assu...",Art. 986. Enquanto não inscritos os atos const...,"Art. 1.793. O direito à sucessão aberta, bem c...",Art. 1.791. A herança defere-se como um todo u...
6,"Art. 1.596. Os filhos, havidos ou não da relaç...","Art. 1.596. Os filhos, havidos ou não da relaç...","Art. 1.658. No regime de comunhão parcial, com...","Art. 1.639. É lícito aos nubentes, antes de ce...",Art. 1.576. A separação judicial põe termo aos...,"Art. 1.596. Os filhos, havidos ou não da relaç..."
7,Art. 1.997. A herança responde pelo pagamento ...,Art. 1.997. A herança responde pelo pagamento ...,"Art. 789. Nos seguros de pessoas, o capital se...","Art. 818. Pelo contrato de fiança, uma pessoa ...",Art. 1.792. O herdeiro não responde por encarg...,Art. 1.791. A herança defere-se como um todo u...
8,Art. 26. Decorrido um ano da arrecadação dos b...,Art. 26. Decorrido um ano da arrecadação dos b...,"Art. 1.824. O herdeiro pode, em ação de petiçã...","Art. 1.824. O herdeiro pode, em ação de petiçã...",Art. 1.813. Quando o herdeiro prejudicar os se...,Art. 1.814. São excluídos da sucessão os herde...
9,"Art. 1.857. Toda pessoa capaz pode dispor, por...","Art. 1.857. Toda pessoa capaz pode dispor, por...",Art. 26. Decorrido um ano da arrecadação dos b...,Art. 1.798. Legitimam-se a suceder as pessoas ...,Art. 1.830. Somente é reconhecido direito suce...,Art. 1.845. São herdeiros necessários os desce...


In [7]:
def loop_fn(df, fn, kwarg=None):
    list_of_outputs = []
    for index, row in tqdm(df.iterrows(), total=len(df)):
        try:
            if kwarg:
                output = fn(row, kwarg)
            else:
                output = fn(row)
            print(output)
        except Exception as e:
            print(f"Error processing row {index}: {e}")
            output = "Error"
        list_of_outputs.append(output)
    return list_of_outputs

In [8]:
class GET_CONTEXT_JSON(BaseModel):
    analise: str = Field(description="Análise da questão, resposta e artigos de contexto.")
    parte_dos_artigos: List[str] = Field(description="Porçao ou trecho do artigo que ajuda a responder a pergunta.")
    artigos_id: List[str] = Field(description="Nome do artigos cujos trechos respondem a pergunta.")
    justificativa_de_cada_trecho: List[str] = Field(description="Razões pelas quais os trechos ajudam a responder a pergunta.")
    informacao_faltante: str = Field(description="Informação faltante que impede a resolução correta da questão, se houver.")
    resposta_obtida: str = Field(description="Resposta obtida pela análise dos trechos selecionados.")
    alternativa_resposta_obtida: str = Field(description="Alternativa da resposta obtida: A, B, C, D ou E.")
    resposta_correta: str = Field(description="Sim, se a resposta obtida for igual a resposta oficial.")


output_parser = JsonOutputParser(pydantic_object=GET_CONTEXT_JSON)

In [9]:
prompt = """
Você é professor de Direito e ávido conhecedor do Código Civil Brasileiro.
Use seus conhecimentos em leis e códigos analisar os itens da questão.

Em seguida selecione trechos dos artigos que melhor resolvem as sentenças da questão.

Irei fornecer uma lista de possíveis artigos do Código Civil e uma outra lista de artigos
citados por estudantes.

Os trechos devem ser usados para CONFIRMAR ou NEGAR as sentenças da questão.

Depois de gerar os trechos, análise se eles ajudam a chegar na resposta correta.
Caso não possibilitem chegar na mesma resposta, indique que informação está faltando.

questão: {questão}
resposta correta: {resposta}
artigos estudantes: {artigos_estudantes}
artigos possíveis: {artigos_da_busca}

Gere somente as informações dos campos do JSON, sem introduções ou formalismos.
Instruções de saída: {format_instructions}
""".strip()

prompt_template = PromptTemplate(
    input_variables=["questão", "resposta", "artigos_estudantes", "artigos_da_busca"],
    template=prompt,
    partial_variables={"format_instructions": output_parser.get_format_instructions()},
)

chain = prompt_template | llama3_70b_temp0 | output_parser

In [15]:
def call_context_generator(sample, rag_column):
    output = chain.invoke(
        {"questão": sample['pergunta'],
         "resposta": sample['resposta'],
         "artigos_estudantes": sample['artigos_completos'],
         "artigos_da_busca": sample[rag_column],
         })
    return output

# test
out = call_context_generator(df.iloc[28], rag_column='rag_assunto_full')
out

{'analise': 'A questão apresenta uma situação de sucessão legítima, onde Mariana falece sem testamento e sem herdeiros necessários. Os irmãos germanos Marcos, Mário, Mirtes e Maitê são chamados à sucessão.',
 'parte_dos_artigos': ['Art. 1.816. São pessoais os efeitos da exclusão; os descendentes do herdeiro excluído sucedem, como se ele morto fosse antes da abertura da sucessão.',
  'Art. 1.811. Ninguém pode suceder, representando herdeiro renunciante. Se, porém, ele for o único legítimo da sua classe, ou se todos os outros da mesma classe renunciarem a herança, poderão os filhos vir à sucessão, por direito próprio, e por cabeça.',
  'Art. 1.853. Na linha transversal, somente se dá o direito de representação em favor dos filhos de irmãos do falecido, quando com irmãos deste concorrerem.'],
 'artigos_id': ['Art. 1.816', 'Art. 1.811', 'Art. 1.853'],
 'justificativa_de_cada_trecho': ['O art. 1.816 explica que os descendentes do herdeiro excluído sucedem como se ele tivesse morrido antes d

In [16]:
list_of_outputs = loop_fn(df, call_context_generator, kwarg='rag_assunto_full')

  3%|▎         | 1/37 [00:23<14:06, 23.51s/it]

{'analise': 'A questão envolve a análise dos direitos do herdeiro aparente de boa-fé em relação à herança e às benfeitorias realizadas no sítio herdado.', 'parte_dos_artigos': ['Art. 1.214. O possuidor de boa-fé tem direito, enquanto ela durar, aos frutos percebidos.', 'Art. 1.219. O possuidor de boa-fé tem direito à indenização das benfeitorias necessárias e úteis, bem como, quanto às voluptuárias, se não lhe forem pagas, a levantá-las, quando o puder sem detrimento da coisa, e poderá exercer o direito de retenção pelo valor das benfeitorias necessárias e úteis.'], 'artigos_id': ['Art. 1.214', 'Art. 1.219'], 'justificativa_de_cada_trecho': ['O art. 1.214 estabelece que o possuidor de boa-fé tem direito aos frutos percebidos enquanto durar a boa-fé.', 'O art. 1.219 estabelece que o possuidor de boa-fé tem direito à indenização das benfeitorias necessárias e úteis, e pode exercer o direito de retenção pelo valor das benfeitorias necessárias e úteis.'], 'informacao_faltante': '', 'respos

  5%|▌         | 2/37 [00:27<06:57, 11.93s/it]

{'analise': 'Análise da questão: A sucessão testamentária é um componente importante do planejamento patrimonial e permite que as pessoas tenham controle sobre o destino de seus bens após a morte.', 'parte_dos_artigos': ['Art. 1.867. Ao cego só se permite o testamento público, que lhe será lido, em voz alta, duas vezes, uma pelo tabelião ou por seu substituto legal, e a outra por uma das testemunhas, designada pelo testador, fazendo-se de tudo circunstanciada menção no testamento.', 'Art. 1.868. O testamento escrito pelo testador, ou por outra pessoa, a seu rogo, e por aquele assinado, será válido se aprovado pelo tabelião ou seu substituto legal, observadas as seguintes formalidades:I - que o testador o entregue ao tabelião em presença de duas testemunhas;II - que o testador declare que aquele é o seu testamento e quer que seja aprovado;III - que o tabelião lavre, desde logo, o auto de aprovação, na presença de duas testemunhas, e o leia, em seguida, ao testador e testemunhas;IV - que

  8%|▊         | 3/37 [00:54<10:41, 18.86s/it]

...

{'analise': 'A legítima dos herdeiros necessários não pode ser incluída no testamento, pois é um direito garantido por lei e não pode ser modificado pelo testador.', 'parte_dos_artigos': ['§ 1 o A legítima dos herdeiros necessários não poderá ser incluída no testamento.'], 'artigos_id': ['Art. 1.857'], 'justificativa_de_cada_trecho': ['Este artigo estabelece que a legítima dos herdeiros necessários não pode ser incluída no testamento, garantindo assim o direito dos herdeiros necessários.'], 'informacao_faltante': '', 'resposta_obtida': 'Certo', 'alternativa_resposta_obtida': 'C', 'resposta_correta': 'Sim'}


 97%|█████████▋| 36/37 [25:35<00:48, 48.78s/it]

{'analise': 'A petição de herança de Renato está prescrita, pois o prazo prescricional é de 10 anos, contados da abertura da sucessão, e Renato ajuizou a ação em 2022, mais de 12 anos após a abertura da sucessão em 2010.', 'parte_dos_artigos': ['Art. 205. A prescrição ocorre em dez anos, quando a lei não lhe haja fixado prazo menor.', 'Art. 1.822. A declaração de vacância da herança não prejudicará os herdeiros que legalmente se habilitarem; mas, decorridos cinco anos da abertura da sucessão, os bens arrecadados passarão ao domínio do Município ou do Distrito Federal, se localizados nas respectivas circunscrições, incorporando-se ao domínio da União quando situados em território federal.'], 'artigos_id': ['Art. 205', 'Art. 1.822'], 'justificativa_de_cada_trecho': ['O art. 205 estabelece o prazo prescricional de 10 anos para a petição de herança.', 'O art. 1.822 estabelece que, decorridos 5 anos da abertura da sucessão, os bens arrecadados passarão ao domínio do Município ou do Distrito

100%|██████████| 37/37 [26:13<00:00, 42.52s/it]

{'analise': 'A questão apresenta uma situação hipotética em que João, nascido em Brasília - DF, viveu toda a sua vida em Penha - SC e morreu em Florianópolis - SC, e sua esposa Ana estava grávida quando ele morreu. A pergunta é se o nascituro tem legitimidade para suceder.', 'parte_dos_artigos': ['Art. 1.798. Legitimam-se a suceder as pessoas nascidas ou já concebidas no momento da abertura da sucessão.'], 'artigos_id': ['Art. 1.798'], 'justificativa_de_cada_trecho': ['Este artigo estabelece que as pessoas concebidas no momento da abertura da sucessão têm legitimidade para suceder.'], 'informacao_faltante': '', 'resposta_obtida': 'Sim, o nascituro tem legitimidade para suceder.', 'alternativa_resposta_obtida': 'C', 'resposta_correta': 'Sim'}


In [19]:
out = call_context_generator(df.iloc[20], rag_column='rag_assunto_full')
out

RateLimitError: Error code: 429 - {'error': {'message': 'Rate limit reached for model `llama3-70b-8192` in organization `org_01hzjxke94e5aawh5x5jztqzy9` on tokens per minute (TPM): Limit 6000, Used 0, Requested 8781. Please try again in 27.81s. Visit https://console.groq.com/docs/rate-limits for more information.', 'type': 'tokens', 'code': 'rate_limit_exceeded'}}

In [ ]:
list_of_outputs[20] = out

In [27]:
copy_20 = df.iloc[20].copy()
copy_20['artigos_completos'] = """'Art. 1.793. O direito à sucessão aberta, bem como o quinhão de que disponha o co-herdeiro, pode ser objeto de cessão por escritura pública.§ 1º Os direitos, conferidos ao herdeiro em conseqüência de substituição ou de direito de acrescer, presumem-se não abrangidos pela cessão feita anteriormente.§ 2º É ineficaz a cessão, pelo co-herdeiro, de seu direito hereditário sobre qualquer bem da herança considerado singularmente.§ 3º Ineficaz é a disposição, sem prévia autorização do juiz da sucessão, por qualquer herdeiro, de bem componente do acervo hereditário, pendente a indivisibilidade.', 'Art. 1.794. O co-herdeiro não poderá ceder a sua quota hereditária a pessoa estranha à sucessão, se outro co-herdeiro a quiser, tanto por tanto.', 'Art. 1.794. O co-herdeiro não poderá ceder a sua quota hereditária a pessoa estranha à sucessão, se outro co-herdeiro a quiser, tanto por tanto.', 'Art. 1.795. O co-herdeiro, a quem não se der conhecimento da cessão, poderá, depositado o preço, haver para si a quota cedida a estranho, se o requerer até cento e oitenta dias após a transmissão.Parágrafo único. Sendo vários os co-herdeiros a exercer a preferência, entre eles se distribuirá o quinhão cedido, na proporção das respectivas quotas hereditárias.', 'Art. 1.795. O co-herdeiro, a quem não se der conhecimento da cessão, poderá, depositado o preço, haver para si a quota cedida a estranho, se o requerer até cento e oitenta dias após a transmissão.Parágrafo único. Sendo vários os co-herdeiros a exercer a preferência, entre eles se distribuirá o quinhão cedido, na proporção das respectivas quotas hereditárias.', 'Art. 1.794. O co-herdeiro não poderá ceder a sua quota hereditária a pessoa estranha à sucessão, se outro co-herdeiro a quiser, tanto por tanto.', 'Art. 1.793. O direito à sucessão aberta, bem como o quinhão de que disponha o co-herdeiro, pode ser objeto de cessão por escritura pública.§ 1º Os direitos, conferidos ao herdeiro em conseqüência de substituição ou de direito de acrescer, presumem-se não abrangidos pela cessão feita anteriormente.§ 2º É ineficaz a cessão, pelo co-herdeiro, de seu direito hereditário sobre qualquer bem da herança considerado singularmente.§ 3º Ineficaz é a disposição, sem prévia autorização do juiz da sucessão, por qualquer herdeiro, de bem componente do acervo hereditário, pendente a indivisibilidade.'"""
copy_20['rag_assunto_full'] = """Art. 47. Obrigam a pessoa jurídica os atos dos administradores, exercidos nos limites de seus poderes definidos no ato constitutivo. Art. 48. Se a pessoa jurídica tiver administração coletiva, as decisões se tomarão pela maioria de votos dos presentes, salvo se o ato constitutivo dispuser de modo diverso. Parágrafo único. Decai em três anos o direito de anular as decisões a que se refere este artigo, quando violarem a lei ou estatuto, ou forem eivadas de erro, dolo, simulação ou fraude. Art. 48-A. As pessoas jurídicas de direito privado, sem prejuízo do previsto em legislação especial e em seus atos constitutivos, poderão realizar suas assembleias gerais por meios eletrônicos, inclusive para os fins do disposto no art. 59, respeitados os direitos previstos de participação e de manifestação. (Incluído pela Medida Provisória nº 1.085, de 2021) Art. 48-A. As pessoas jurídicas de direito privado, sem prejuízo do previsto em legislação especial e em seus atos constitutivos, poderão realizar suas assembleias gerais por meio eletrônico, inclusive para os fins do disposto no art. 59 deste Código, respeitados os direitos previstos de participação e de manifestação. (Incluído pela Lei nº 14.382, de 2022) Art. 49. Se a administração da pessoa jurídica vier a faltar, o juiz, a requerimento de qualquer interessado, nomear-lhe-á administrador provisório. Art. 49-A. A pessoa jurídica não se confunde com os seus sócios, associados, instituidores ou administradores. (Incluído pela Lei nº 13.874, de 2019) Parágrafo único. A autonomia patrimonial das pessoas jurídicas é um instrumento lícito de alocação e segregação de riscos, estabelecido pela lei com a finalidade de estimular empreendimentos, para a geração de empregos, tributo, renda e inovação em benefício de todos. (Incluído pela Lei nº 13.874, de 2019) Art. 50. Em caso de abuso da personalidade jurídica, caracterizado pelo desvio de finalidade, ou pela confusão patrimonial, pode o juiz decidir, a requerimento da parte, ou do Ministério Público quando lhe couber intervir no processo, que os efeitos de certas e determinadas relações de obrigações sejam estendidos aos bens particulares dos administradores ou sócios da pessoa jurídica. Art. 50. Em caso de abuso da personalidade jurídica, caracterizado pelo desvio de finalidade ou pela confusão patrimonial, pode o juiz, a requerimento da parte, ou do Ministério Público quando lhe couber intervir no processo, desconsiderá-la para que os efeitos de certas e determinadas relações de obrigações sejam estendidos aos bens particulares de administradores ou de sócios da pessoa jurídica beneficiados direta ou indiretamente pelo abuso. (Redação dada pela Medida Provisória nº 881, de 2019) § 1º Para fins do disposto neste artigo, desvio de finalidade é a utilização dolosa da pessoa jurídica com o propósito de lesar credores e para a prática de atos ilícitos de qualquer natureza. (Incluído pela Medida Provisória nº 881, de 2019) § 2º Entende-se por confusão patrimonial a ausência de separação de fato entre os patrimônios, caracterizada por: (Incluído pela Medida Provisória nº 881, de 2019) I - cumprimento repetitivo pela sociedade de obrigações do sócio ou do administrador ou vice-versa; (Incluído pela Medida Provisória nº 881, de 2019) II - transferência de ativos ou de passivos sem efetivas contraprestações, exceto o de valor proporcionalmente insignificante; e (Incluído pela Medida Provisória nº 881, de 2019) III - outros atos de descumprimento da autonomia patrimonial. (Incluído pela Medida Provisória nº 881, de 2019) § 3º O disposto no caput e nos § 1º e § 2º também se aplica à extensão das obrigações de sócios ou de administradores à pessoa jurídica. (Incluído pela Medida Provisória nº 881, de 2019) § 4º A mera existência de grupo econômico sem a presença dos requisitos de que trata o caput não autoriza a desconsideração da personalidade da pessoa jurídica. (Incluído pela Medida Provisória nº 881, de 2019) § 5º Não constitui desvio de finalidade a mera expansão ou a alteração da finalidade original da atividade econômica específica da pessoa jurídica. (Incluído pela Medida Provisória nº 881, de 2019) Art. 50. Em caso de abuso da personalidade jurídica, caracterizado pelo desvio de finalidade ou pela confusão patrimonial, pode o juiz, a requerimento da parte, ou do Ministério Público quando lhe couber intervir no processo, desconsiderá-la para que os efeitos de certas e determinadas relações de obrigações sejam estendidos aos bens particulares de administradores ou de sócios da pessoa jurídica beneficiados direta ou indiretamente pelo abuso. (Redação dada pela Lei nº 13.874, de 2019) § 1º Para os fins do disposto neste artigo, desvio de finalidade é a utilização da pessoa jurídica com o propósito de lesar credores e para a prática de atos ilícitos de qualquer natureza. (Incluído pela Lei nº 13.874, de 2019) § 2º Entende-se por confusão patrimonial a ausência de separação de fato entre os patrimônios, caracterizada por: (Incluído pela Lei nº 13.874, de 2019) I - cumprimento repetitivo pela sociedade de obrigações do sócio ou do administrador ou vice-versa; (Incluído pela Lei nº 13.874, de 2019) II - transferência de ativos ou de passivos sem efetivas contraprestações, exceto os de valor proporcionalmente insignificante; e (Incluído pela Lei nº 13.874, de 2019) III - outros atos de descumprimento da autonomia patrimonial. (Incluído pela Lei nº 13.874, de 2019) § 3º O disposto no caput e nos §§ 1º e 2º deste artigo também se aplica à extensão das obrigações de sócios ou de administradores à pessoa jurídica. (Incluído pela Lei nº 13.874, de 2019) § 4º A mera existência de grupo econômico sem a presença dos requisitos de que trata o caput deste artigo não autoriza a desconsideração da personalidade da pessoa jurídica. (Incluído pela Lei nº 13.874, de 2019) § 5º Não constitui desvio de finalidade a mera expansão ou a alteração da finalidade original da atividade econômica específica da pessoa jurídica. (Incluído pela Lei nº 13.874, de 2019) Art. 51. Nos casos de dissolução da pessoa jurídica ou cassada a autorização para seu funcionamento, ela subsistirá para os fins de liquidação, até que esta se conclua. § 1º Far-se-á, no registro onde a pessoa jurídica estiver inscrita, a averbação de sua dissolução. § 2º As disposições para a liquidação das sociedades aplicam-se, no que couber, às demais pessoas jurídicas de direito privado. § 3º Encerrada a liquidação, promover-se-á o cancelamento da inscrição da pessoa jurídica. Art. 52. Aplica-se às pessoas jurídicas, no que couber, a proteção dos direitos da personalidade. Art. 26. Decorrido um ano da arrecadação dos bens do ausente, ou, se ele deixou representante ou procurador, em se passando três anos, poderão os interessados requerer que se declare a ausência e se abra provisoriamente a sucessão. Art. 27. Para o efeito previsto no artigo anterior, somente se consideram interessados: I - o cônjuge não separado judicialmente; II - os herdeiros presumidos, legítimos ou testamentários; III - os que tiverem sobre os bens do ausente direito dependente de sua morte; IV - os credores de obrigações vencidas e não pagas. Art. 28. A sentença que determinar a abertura da sucessão provisória só produzirá efeito cento e oitenta dias depois de publicada pela imprensa; mas, logo que passe em julgado, proceder-se-á à abertura do testamento, se houver, e ao inventário e partilha dos bens, como se o ausente fosse falecido. § 1º Findo o prazo a que se refere o art. 26 , e não havendo interessados na sucessão provisória, cumpre ao Ministério Público requerê-la ao juízo competente. § 2º Não comparecendo herdeiro ou interessado para requerer o inventário até trinta dias depois de passar em julgado a sentença que mandar abrir a sucessão provisória, proceder-se-á à arrecadação dos bens do ausente pela forma estabelecida nos arts. 1.819 a 1.823 . Art. 29. Antes da partilha, o juiz, quando julgar conveniente, ordenará a conversão dos bens móveis, sujeitos a deterioração ou a extravio, em imóveis ou em títulos garantidos pela União. Art. 30. Os herdeiros, para se imitirem na posse dos bens do ausente, darão garantias da restituição deles, mediante penhores ou hipotecas equivalentes aos quinhões respectivos. § 1º Aquele que tiver direito à posse provisória, mas não puder prestar a garantia exigida neste artigo, será excluído, mantendo-se os bens que lhe deviam caber sob a administração do curador, ou de outro herdeiro designado pelo juiz, e que preste essa garantia. § 2º Os ascendentes, os descendentes e o cônjuge, uma vez provada a sua qualidade de herdeiros, poderão, independentemente de garantia, entrar na posse dos bens do ausente. Art. 31. Os imóveis do ausente só se poderão alienar, não sendo por desapropriação, ou hipotecar, quando o ordene o juiz, para lhes evitar a ruína. Art. 35. Se durante a posse provisória se provar a época exata do falecimento do ausente, considerar-se-á, nessa data, aberta a sucessão em favor dos herdeiros, que o eram àquele tempo. Art. 36. Se o ausente aparecer, ou se lhe provar a existência, depois de estabelecida a posse provisória, cessarão para logo as vantagens dos sucessores nela imitidos, ficando, todavia, obrigados a tomar as medidas assecuratórias precisas, até a entrega dos bens a seu dono. Art. 972. Podem exercer a atividade de empresário os que estiverem em pleno gozo da capacidade civil e não forem legalmente impedidos. Art. 973. A pessoa legalmente impedida de exercer atividade própria de empresário, se a exercer, responderá pelas obrigações contraídas. Art. 974. Poderá o incapaz, por meio de representante ou devidamente assistido, continuar a empresa antes exercida por ele enquanto capaz, por seus pais ou pelo autor de herança. §1º Nos casos deste artigo, precederá autorização judicial, após exame das circunstâncias e dos riscos da empresa, bem como da conveniência em continuá-la, podendo a autorização ser revogada pelo juiz, ouvidos os pais, tutores ou representantes legais do menor ou do interdito, sem prejuízo dos direitos adquiridos por terceiros. §2º Não ficam sujeitos ao resultado da empresa os bens que o incapaz já possuía, ao tempo da sucessão ou da interdição, desde que estranhos ao acervo daquela, devendo tais fatos constar do alvará que conceder a autorização. §3º O Registro Público de Empresas Mercantis a cargo das Juntas Comerciais deverá registrar contratos ou alterações contratuais de sociedade que envolva sócio incapaz, desde que atendidos, de forma conjunta, os seguintes pressupostos: (Incluído pela Lei nº 12.399, de 2011) I – o sócio incapaz não pode exercer a administração da sociedade; (Incluído pela Lei nº 12.399, de 2011) II – o capital social deve ser totalmente integralizado; (Incluído pela Lei nº 12.399, de 2011) III – o sócio relativamente incapaz deve ser assistido e o absolutamente incapaz deve ser representado por seus representantes legais. (Incluído pela Lei nº 12.399, de 2011) Art. 975. Se o representante ou assistente do incapaz for pessoa que, por disposição de lei, não puder exercer atividade de empresário, nomeará, com a aprovação do juiz, um ou mais gerentes. §1º Do mesmo modo será nomeado gerente em todos os casos em que o juiz entender ser conveniente. §2º A aprovação do juiz não exime o representante ou assistente do menor ou do interdito da responsabilidade pelos atos dos gerentes nomeados. Art. 976. A prova da emancipação e da autorização do incapaz, nos casos do art. 974 , e a de eventual revogação desta, serão inscritas ou averbadas no Registro Público de Empresas Mercantis. Parágrafo único. O uso da nova firma caberá, conforme o caso, ao gerente; ou ao representante do incapaz; ou a este, quando puder ser autorizado. Art. 977. Faculta-se aos cônjuges contratar sociedade, entre si ou com terceiros, desde que não tenham casado no regime da comunhão universal de bens, ou no da separação obrigatória. Art. 978. O empresário casado pode, sem necessidade de outorga conjugal, qualquer que seja o regime de bens, alienar os imóveis que integrem o patrimônio da empresa ou gravá-los de ônus real. Art. 979. Além de no Registro Civil, serão arquivados e averbados, no Registro Público de Empresas Mercantis, os pactos e declarações antenupciais do empresário, o título de doação, herança, ou legado, de bens clausulados de incomunicabilidade ou inalienabilidade. Art. 980. A sentença que decretar ou homologar a separação judicial do empresário e o ato de reconciliação não podem ser opostos a terceiros, antes de arquivados e averbados no Registro Público de Empresas Mercantis. TÍTULO I-A (Incluído pela Lei nº 12.441, de 2011) (Vigência) DA EMPRESA INDIVIDUAL DE RESPONSABILIDADE LIMITADA (Revogado Pela Medida Provisória nº 1.085, de 2021) (Revogado pela Lei nº 14.382, de 2022) Art. 980-A. A empresa individual de responsabilidade limitada será constituída por uma única pessoa titular da totalidade do capital social, devidamente integralizado, que não será inferior a 100 (cem) vezes o maior salário-mínimo vigente no País. (Incluído pela Lei nº 12.441, de 2011) (Vigência) (Revogado Pela Medida Provisória nº 1.085, de 2021) (Revogado pela Lei nº 14.382, de 2022) §1º O nome empresarial deverá ser formado pela inclusão da expressão " EIRELI " após a firma ou a denominação social da empresa individual de responsabilidade limitada. (Incluído pela Lei nº 12.441, de 2011) (Vigência) (Revogado Pela Medida Provisória nº 1.085, de 2021) (Revogado pela Lei nº 14.382, de 2022) §2º A pessoa natural que constituir empresa individual de responsabilidade limitada somente poderá figurar em uma única empresa dessa modalidade. (Incluído pela Lei nº 12.441, de 2011) (Vigência) (Revogado Pela Medida Provisória nº 1.085, de 2021) (Revogado pela Lei nº 14.382, de 2022) §3º A empresa individual de responsabilidade limitada também poderá resultar da concentração das quotas de outra modalidade societária num único sócio, independentemente das razões que motivaram tal concentração. (Incluído pela Lei nº 12.441, de 2011) (Vigência) (Revogado Pela Medida Provisória nº 1.085, de 2021) (Revogado pela Lei nº 14.382, de 2022) §4º ( VETADO) . (Incluído pela Lei nº 12.441, de 2011) (Vigência) (Revogado Pela Medida Provisória nº 1.085, de 2021) (Revogado pela Lei nº 14.382, de 2022) §5º Poderá ser atribuída à empresa individual de responsabilidade limitada constituída para a prestação de serviços de qualquer natureza a remuneração decorrente da cessão de direitos patrimoniais de autor ou de imagem, nome, marca ou voz de que seja detentor o titular da pessoa jurídica, vinculados à atividade profissional. (Incluído pela Lei nº 12.441, de 2011) (Vigência) (Revogado Pela Medida Provisória nº 1.085, de 2021) (Revogado pela Lei nº 14.382, de 2022) § 6º Aplicam-se à empresa individual de responsabilidade limitada, no que couber, as regras previstas para as sociedades limitadas. (Incluído pela Lei nº 12.441, de 2011) (Vigência) (Revogado Pela Medida Provisória nº 1.085, de 2021) (Revogado pela Lei nº 14.382, de 2022) § 7º Somente o patrimônio social da empresa responderá pelas dívidas da empresa individual de responsabilidade limitada, hipótese em que não se confundirá, em qualquer situação, com o patrimônio do titular que a constitui, ressalvados os casos de fraude. (Incluído pela Medida Provisória nº 881, de 2019) §7º Somente o patrimônio social da empresa responderá pelas dívidas da empresa individual de responsabilidade limitada, hipótese em que não se confundirá, em qualquer situação, com o patrimônio do titular que a constitui, ressalvados os casos de fraude. (Incluído pela Lei nº 13.874, de 2019) (Revogado Pela Medida Provisória nº 1.085, de 2021) (Revogado pela Lei nº 14.382, de 2022) Art. 1.639. É lícito aos nubentes, antes de celebrado o casamento, estipular, quanto aos seus bens, o que lhes aprouver. § 1 o O regime de bens entre os cônjuges começa a vigorar desde a data do casamento. § 2 o É admissível alteração do regime de bens, mediante autorização judicial em pedido motivado de ambos os cônjuges, apurada a procedência das razões invocadas e ressalvados os direitos de terceiros. Art. 1.640. Não havendo convenção, ou sendo ela nula ou ineficaz, vigorará, quanto aos bens entre os cônjuges, o regime da comunhão parcial. Parágrafo único. Poderão os nubentes, no processo de habilitação, optar por qualquer dos regimes que este código regula. Quanto à forma, reduzir-se-á a termo a opção pela comunhão parcial, fazendo-se o pacto antenupcial por escritura pública, nas demais escolhas."""

In [28]:
out = call_context_generator(copy_20, rag_column='rag_assunto_full') #df.iloc[22]
out

{'analise': 'A cessão de direitos hereditários é ato permitido aos herdeiros, em conformidade com o disposto no Código Civil.',
 'parte_dos_artigos': ['Art. 1.793. O direito à sucessão aberta, bem como o quinhão de que disponha o co-herdeiro, pode ser objeto de cessão por escritura pública.',
  'Art. 1.794. O co-herdeiro não poderá ceder a sua quota hereditária a pessoa estranha à sucessão, se outro co-herdeiro a quiser, tanto por tanto.',
  'Art. 1.795. O co-herdeiro, a quem não se der conhecimento da cessão, poderá, depositado o preço, haver para si a quota cedida a estranho, se o requerer até cento e oitenta dias após a transmissão.'],
 'artigos_id': ['Art. 1.793', 'Art. 1.794', 'Art. 1.795'],
 'justificativa_de_cada_trecho': ['Este artigo permite a cessão de direitos hereditários por escritura pública.',
  'Este artigo impede a cessão de quota hereditária a pessoa estranha à sucessão se outro co-herdeiro a quiser.',
  'Este artigo permite ao co-herdeiro que não teve conhecimento da

In [29]:
list_of_outputs[20] = out

In [30]:
copy_22 = df.iloc[22].copy()
copy_22["rag_assunto_full"] = """Art. 43. As pessoas jurídicas de direito público interno são civilmente responsáveis por atos dos seus agentes que nessa qualidade causem danos a terceiros, ressalvado direito regressivo contra os causadores do dano, se houver, por parte destes, culpa ou dolo. Art. 44. São pessoas jurídicas de direito privado: I - as associações; II - as sociedades; III - as fundações. IV - as organizações religiosas; (Incluído pela Lei nº 10.825, de 22.12.2003) V - os partidos políticos. (Incluído pela Lei nº 10.825, de 22.12.2003) VI - as empresas individuais de responsabilidade limitada. (Incluído pela Lei nº 12.441, de 2011) (Vigência) (Revogado Pela Medida Provisória nº 1.085, de 2021) (Revogado pela Lei nº 14.382, de 2022) § 1º São livres a criação, a organização, a estruturação interna e o funcionamento das organizações religiosas, sendo vedado ao poder público negar-lhes reconhecimento ou registro dos atos constitutivos e necessários ao seu funcionamento. (Incluído pela Lei nº 10.825, de 22.12.2003) § 2º As disposições concernentes às associações aplicam-se subsidiariamente às sociedades que são objeto do Livro II da Parte Especial deste Código . (Incluído pela Lei nº 10.825, de 22.12.2003) § 3º Os partidos políticos serão organizados e funcionarão conforme o disposto em lei específica. (Incluído pela Lei nº 10.825, de 22.12.2003) Art. 45. Começa a existência legal das pessoas jurídicas de direito privado com a inscrição do ato constitutivo no respectivo registro, precedida, quando necessário, de autorização ou aprovação do Poder Executivo, averbando-se no registro todas as alterações por que passar o ato constitutivo. Parágrafo único. Decai em três anos o direito de anular a constituição das pessoas jurídicas de direito privado, por defeito do ato respectivo, contado o prazo da publicação de sua inscrição no registro. Art. 46. O registro declarará: I - a denominação, os fins, a sede, o tempo de duração e o fundo social, quando houver; II - o nome e a individualização dos fundadores ou instituidores, e dos diretores; III - o modo por que se administra e representa, ativa e passivamente, judicial e extrajudicialmente; IV - se o ato constitutivo é reformável no tocante à administração, e de que modo; V - se os membros respondem, ou não, subsidiariamente, pelas obrigações sociais; VI - as condições de extinção da pessoa jurídica e o destino do seu patrimônio, nesse caso. Art. 47. Obrigam a pessoa jurídica os atos dos administradores, exercidos nos limites de seus poderes definidos no ato constitutivo. Art. 48. Se a pessoa jurídica tiver administração coletiva, as decisões se tomarão pela maioria de votos dos presentes, salvo se o ato constitutivo dispuser de modo diverso. Parágrafo único. Decai em três anos o direito de anular as decisões a que se refere este artigo, quando violarem a lei ou estatuto, ou forem eivadas de erro, dolo, simulação ou fraude. Art. 48-A. As pessoas jurídicas de direito privado, sem prejuízo do previsto em legislação especial e em seus atos constitutivos, poderão realizar suas assembleias gerais por meios eletrônicos, inclusive para os fins do disposto no art. 59, respeitados os direitos previstos de participação e de manifestação. (Incluído pela Medida Provisória nº 1.085, de 2021) Art. 48-A. As pessoas jurídicas de direito privado, sem prejuízo do previsto em legislação especial e em seus atos constitutivos, poderão realizar suas assembleias gerais por meio eletrônico, inclusive para os fins do disposto no art. 59 deste Código, respeitados os direitos previstos de participação e de manifestação. (Incluído pela Lei nº 14.382, de 2022) Art. 49. Se a administração da pessoa jurídica vier a faltar, o juiz, a requerimento de qualquer interessado, nomear-lhe-á administrador provisório. Art. 49-A. A pessoa jurídica não se confunde com os seus sócios, associados, instituidores ou administradores. (Incluído pela Lei nº 13.874, de 2019) Parágrafo único. A autonomia patrimonial das pessoas jurídicas é um instrumento lícito de alocação e segregação de riscos, estabelecido pela lei com a finalidade de estimular empreendimentos, para a geração de empregos, tributo, renda e inovação em benefício de todos. (Incluído pela Lei nº 13.874, de 2019) Art. 50. Em caso de abuso da personalidade jurídica, caracterizado pelo desvio de finalidade, ou pela confusão patrimonial, pode o juiz decidir, a requerimento da parte, ou do Ministério Público quando lhe couber intervir no processo, que os efeitos de certas e determinadas relações de obrigações sejam estendidos aos bens particulares dos administradores ou sócios da pessoa jurídica. Art. 50. Em caso de abuso da personalidade jurídica, caracterizado pelo desvio de finalidade ou pela confusão patrimonial, pode o juiz, a requerimento da parte, ou do Ministério Público quando lhe couber intervir no processo, desconsiderá-la para que os efeitos de certas e determinadas relações de obrigações sejam estendidos aos bens particulares de administradores ou de sócios da pessoa jurídica beneficiados direta ou indiretamente pelo abuso. (Redação dada pela Medida Provisória nº 881, de 2019) § 1º Para fins do disposto neste artigo, desvio de finalidade é a utilização dolosa da pessoa jurídica com o propósito de lesar credores e para a prática de atos ilícitos de qualquer natureza. (Incluído pela Medida Provisória nº 881, de 2019) § 2º Entende-se por confusão patrimonial a ausência de separação de fato entre os patrimônios, caracterizada por: (Incluído pela Medida Provisória nº 881, de 2019) I - cumprimento repetitivo pela sociedade de obrigações do sócio ou do administrador ou vice-versa; (Incluído pela Medida Provisória nº 881, de 2019) II - transferência de ativos ou de passivos sem efetivas contraprestações, exceto o de valor proporcionalmente insignificante; e (Incluído pela Medida Provisória nº 881, de 2019) III - outros atos de descumprimento da autonomia patrimonial. (Incluído pela Medida Provisória nº 881, de 2019) § 3º O disposto no caput e nos § 1º e § 2º também se aplica à extensão das obrigações de sócios ou de administradores à pessoa jurídica. (Incluído pela Medida Provisória nº 881, de 2019) § 4º A mera existência de grupo econômico sem a presença dos requisitos de que trata o caput não autoriza a desconsideração da personalidade da pessoa jurídica. (Incluído pela Medida Provisória nº 881, de 2019) § 5º Não constitui desvio de finalidade a mera expansão ou a alteração da finalidade original da atividade econômica específica da pessoa jurídica. (Incluído pela Medida Provisória nº 881, de 2019) Art. 50. Em caso de abuso da personalidade jurídica, caracterizado pelo desvio de finalidade ou pela confusão patrimonial, pode o juiz, a requerimento da parte, ou do Ministério Público quando lhe couber intervir no processo, desconsiderá-la para que os efeitos de certas e determinadas relações de obrigações sejam estendidos aos bens particulares de administradores ou de sócios da pessoa jurídica beneficiados direta ou indiretamente pelo abuso. (Redação dada pela Lei nº 13.874, de 2019) § 1º Para os fins do disposto neste artigo, desvio de finalidade é a utilização da pessoa jurídica com o propósito de lesar credores e para a prática de atos ilícitos de qualquer natureza. (Incluído pela Lei nº 13.874, de 2019) § 2º Entende-se por confusão patrimonial a ausência de separação de fato entre os patrimônios, caracterizada por: (Incluído pela Lei nº 13.874, de 2019) I - cumprimento repetitivo pela sociedade de obrigações do sócio ou do administrador ou vice-versa; (Incluído pela Lei nº 13.874, de 2019) II - transferência de ativos ou de passivos sem efetivas contraprestações, exceto os de valor proporcionalmente insignificante; e (Incluído pela Lei nº 13.874, de 2019) III - outros atos de descumprimento da autonomia patrimonial. (Incluído pela Lei nº 13.874, de 2019) § 3º O disposto no caput e nos §§ 1º e 2º deste artigo também se aplica à extensão das obrigações de sócios ou de administradores à pessoa jurídica. (Incluído pela Lei nº 13.874, de 2019) § 4º A mera existência de grupo econômico sem a presença dos requisitos de que trata o caput deste artigo não autoriza a desconsideração da personalidade da pessoa jurídica. (Incluído pela Lei nº 13.874, de 2019) § 5º Não constitui desvio de finalidade a mera expansão ou a alteração da finalidade original da atividade econômica específica da pessoa jurídica. (Incluído pela Lei nº 13.874, de 2019) Art. 51. Nos casos de dissolução da pessoa jurídica ou cassada a autorização para seu funcionamento, ela subsistirá para os fins de liquidação, até que esta se conclua. § 1º Far-se-á, no registro onde a pessoa jurídica estiver inscrita, a averbação de sua dissolução. § 2º As disposições para a liquidação das sociedades aplicam-se, no que couber, às demais pessoas jurídicas de direito privado. § 3º Encerrada a liquidação, promover-se-á o cancelamento da inscrição da pessoa jurídica. Art. 52. Aplica-se às pessoas jurídicas, no que couber, a proteção dos direitos da personalidade. Art. 474. A cláusula resolutiva expressa opera de pleno direito; a tácita depende de interpelação judicial. Art. 475. A parte lesada pelo inadimplemento pode pedir a resolução do contrato, se não preferir exigir-lhe o cumprimento, cabendo, em qualquer dos casos, indenização por perdas e danos. Art. 212. Salvo o negócio a que se impõe forma especial, o fato jurídico pode ser provado mediante: I - confissão; II - documento; III - testemunha; IV - presunção; V - perícia. Art. 213. Não tem eficácia a confissão se provém de quem não é capaz de dispor do direito a que se referem os fatos confessados. Parágrafo único. Se feita a confissão por um representante, somente é eficaz nos limites em que este pode vincular o representado. Art. 214. A confissão é irrevogável, mas pode ser anulada se decorreu de erro de fato ou de coação. Art. 215. A escritura pública, lavrada em notas de tabelião, é documento dotado de fé pública, fazendo prova plena. § 1º Salvo quando exigidos por lei outros requisitos, a escritura pública deve conter: I - data e local de sua realização; II - reconhecimento da identidade e capacidade das partes e de quantos hajam comparecido ao ato, por si, como representantes, intervenientes ou testemunhas; III - nome, nacionalidade, estado civil, profissão, domicílio e residência das partes e demais comparecentes, com a indicação, quando necessário, do regime de bens do casamento, nome do outro cônjuge e filiação; IV - manifestação clara da vontade das partes e dos intervenientes; V - referência ao cumprimento das exigências legais e fiscais inerentes à legitimidade do ato; VI - declaração de ter sido lida na presença das partes e demais comparecentes, ou de que todos a leram; VII - assinatura das partes e dos demais comparecentes, bem como a do tabelião ou seu substituto legal, encerrando o ato. § 2º Se algum comparecente não puder ou não souber escrever, outra pessoa capaz assinará por ele, a seu rogo. § 3º A escritura será redigida na língua nacional. § 4º Se qualquer dos comparecentes não souber a língua nacional e o tabelião não entender o idioma em que se expressa, deverá comparecer tradutor público para servir de intérprete, ou, não o havendo na localidade, outra pessoa capaz que, a juízo do tabelião, tenha idoneidade e conhecimento bastantes. § 5º Se algum dos comparecentes não for conhecido do tabelião, nem puder identificar-se por documento, deverão participar do ato pelo menos duas testemunhas que o conheçam e atestem sua identidade. Art. 216. Farão a mesma prova que os originais as certidões textuais de qualquer peça judicial, do protocolo das audiências, ou de outro qualquer livro a cargo do escrivão, sendo extraídas por ele, ou sob a sua vigilância, e por ele subscritas, assim como os traslados de autos, quando por outro escrivão consertados. Art. 217. Terão a mesma força probante os traslados e as certidões, extraídos por tabelião ou oficial de registro, de instrumentos ou documentos lançados em suas notas. Art. 218. Os traslados e as certidões considerar-se-ão instrumentos públicos, se os originais se houverem produzido em juízo como prova de algum ato. Art. 219. As declarações constantes de documentos assinados presumem-se verdadeiras em relação aos signatários. Parágrafo único. Não tendo relação direta, porém, com as disposições principais ou com a legitimidade das partes, as declarações enunciativas não eximem os interessados em sua veracidade do ônus de prová-las. Art. 220. A anuência ou a autorização de outrem, necessária à validade de um ato, provar-se-á do mesmo modo que este, e constará, sempre que se possa, do próprio instrumento. Art. 221. O instrumento particular, feito e assinado, ou somente assinado por quem esteja na livre disposição e administração de seus bens, prova as obrigações convencionais de qualquer valor; mas os seus efeitos, bem como os da cessão, não se operam, a respeito de terceiros, antes de registrado no registro público. Parágrafo único. A prova do instrumento particular pode suprir-se pelas outras de caráter legal. Art. 222. O telegrama, quando lhe for contestada a autenticidade, faz prova mediante conferência com o original assinado. Art. 223. A cópia fotográfica de documento, conferida por tabelião de notas, valerá como prova de declaração da vontade, mas, impugnada sua autenticidade, deverá ser exibido o original. Parágrafo único. A prova não supre a ausência do título de crédito, ou do original, nos casos em que a lei ou as circunstâncias condicionarem o exercício do direito à sua exibição. Art. 224. Os documentos redigidos em língua estrangeira serão traduzidos para o português para ter efeitos legais no País. Art. 225. As reproduções fotográficas, cinematográficas, os registros fonográficos e, em geral, quaisquer outras reproduções mecânicas ou eletrônicas de fatos ou de coisas fazem prova plena destes, se a parte, contra quem forem exibidos, não lhes impugnar a exatidão. Art. 226. Os livros e fichas dos empresários e sociedades provam contra as pessoas a que pertencem, e, em seu favor, quando, escriturados sem vício extrínseco ou intrínseco, forem confirmados por outros subsídios. Parágrafo único. A prova resultante dos livros e fichas não é bastante nos casos em que a lei exige escritura pública, ou escrito particular revestido de requisitos especiais, e pode ser ilidida pela comprovação da falsidade ou inexatidão dos lançamentos. Art. 227. Salvo os casos expressos, a prova exclusivamente testemunhal só se admite nos negócios jurídicos cujo valor não ultrapasse o décuplo do maior salário mínimo vigente no País ao tempo em que foram celebrados. (Revogado pela Lei n º 13.105, de 2015) (Vigência) Parágrafo único. Qualquer que seja o valor do negócio jurídico, a prova testemunhal é admissível como subsidiária ou complementar da prova por escrito. Art. 228. Não podem ser admitidos como testemunhas: I - os menores de dezesseis anos; II - aqueles que, por enfermidade ou retardamento mental, não tiverem discernimento para a prática dos atos da vida civil; III - os cegos e surdos, quando a ciência do fato que se quer provar dependa dos sentidos que lhes faltam; II - ( Revogado); (Redação dada pela Lei nº 13.146, de 2015) (Vigência) III - (Revogado); (Redação dada pela Lei nº 13.146, de 2015) (Vigência) IV - o interessado no litígio, o amigo íntimo ou o inimigo capital das partes; V - os cônjuges, os ascendentes, os descendentes e os colaterais, até o terceiro grau de alguma das partes, por consangüinidade, ou afinidade. § 1º Para a prova de fatos que só elas conheçam, pode o juiz admitir o depoimento das pessoas a que se refere este artigo. § 2º A pessoa com deficiência poderá testemunhar em igualdade de condições com as demais pessoas, sendo-lhe assegurados todos os recursos de tecnologia assistiva. (Incluído pela Lei nº 13.146, de 2015) (Vigência) Art. 229. Ninguém pode ser obrigado a depor sobre fato: (Revogado pela Lei n º 13.105, de 2015) (Vigência) I - a cujo respeito, por estado ou profissão, deva guardar segredo; (Revogado pela Lei n º 13.105, de 2015) (Vigência) II - a que não possa responder sem desonra própria, de seu cônjuge, parente em grau sucessível, ou amigo íntimo; (Revogado pela Lei n º 13.105, de 2015) (Vigência) III - que o exponha, ou às pessoas referidas no inciso antecedente, a perigo de vida, de demanda, ou de dano patrimonial imediato. (Revogado pela Lei n º 13.105, de 2015) (Vigência) Art. 230. As presunções, que não as legais, não se admitem nos casos em que a lei exclui a prova testemunhal. (Revogado pela Lei n º 13.105, de 2015) (Vigência) Art. 231. Aquele que se nega a submeter-se a exame médico necessário não poderá aproveitar-se de sua recusa. Art. 232. A recusa à perícia médica ordenada pelo juiz poderá suprir a prova que se pretendia obter com o exame. Art. 189. Violado o direito, nasce para o titular a pretensão, a qual se extingue, pela prescrição, nos prazos a que aludem os arts. 205 e 206 . Art. 190. A exceção prescreve no mesmo prazo em que a pretensão. Art. 191. A renúncia da prescrição pode ser expressa ou tácita, e só valerá, sendo feita, sem prejuízo de terceiro, depois que a prescrição se consumar; tácita é a renúncia quando se presume de fatos do interessado, incompatíveis com a prescrição. Art. 192. Os prazos de prescrição não podem ser alterados por acordo das partes. Art. 193. A prescrição pode ser alegada em qualquer grau de jurisdição, pela parte a quem aproveita"""

In [31]:
out = call_context_generator(copy_22, rag_column='rag_assunto_full') #df.iloc[22]
out

{'analise': 'A questão apresentada refere-se à exclusão de um herdeiro por indignidade, especificamente no caso de Pedro, que praticou crime contra a honra do companheiro de Ana pouco antes do falecimento da autora da herança.',
 'parte_dos_artigos': ['Art. 1.816. São pessoais os efeitos da exclusão; os descendentes do herdeiro excluído sucedem, como se ele morto fosse antes da abertura da sucessão.'],
 'artigos_id': ['Art. 1.816'],
 'justificativa_de_cada_trecho': ['Este artigo estabelece que os efeitos da exclusão de um herdeiro são pessoais e que os descendentes do herdeiro excluído sucedem como se ele tivesse morrido antes da abertura da sucessão.'],
 'informacao_faltante': '',
 'resposta_obtida': 'E',
 'alternativa_resposta_obtida': 'E) Caso Pedro seja excluído da sucessão por indignidade, seus herdeiros sucederão, como se Pedro fosse morto antes da abertura da sucessão de Ana.',
 'resposta_correta': 'Sim'}

In [32]:
list_of_outputs[22] = out

In [22]:
df.loc[[20, 22], ['rag_assunto_full', 'pergunta', 'resposta', 'artigos_completos']]

,rag_assunto_full,pergunta,resposta,artigos_completos
20,Art. 40. As pessoas jurídicas são de direito p...,A cessão de direitos hereditários é ato permit...,B,"['Art. 1.793. O direito à sucessão aberta, bem..."
22,Art. 40. As pessoas jurídicas são de direito p...,"Ana faleceu deixando dois filhos, Pedro e Paul...",E,['Art. 1.814. São excluídos da sucessão os her...


In [33]:
df_rag_assunto_full = pd.DataFrame(list_of_outputs)
df_rag_assunto_full.to_csv("df_rag_assunto_full.csv", index=False)

In [34]:
df_rag_assunto_full

,analise,parte_dos_artigos,artigos_id,justificativa_de_cada_trecho,informacao_faltante,resposta_obtida,alternativa_resposta_obtida,resposta_correta
0,A questão envolve a análise dos direitos do he...,[Art. 1.214. O possuidor de boa-fé tem direito...,"[Art. 1.214, Art. 1.219]",[O art. 1.214 estabelece que o possuidor de bo...,,"C) todos os frutos percebidos até a citação, b...",C,Sim
1,Análise da questão: A sucessão testamentária é...,[Art. 1.867. Ao cego só se permite o testament...,"[Art. 1.867, Art. 1.868, Art. 1.880]",[I - O art. 1.867 confirma que ao cego só se p...,,B) I e IV.,B,Sim
2,A questão apresenta um caso de testamento feit...,[Art. 1.880. O testamento particular pode ser ...,"[Art. 1.880, Art. 1.858, Art. 1.860]",[O artigo 1.880 permite que o testamento seja ...,,O testamento pode ser escrito em língua estran...,E,Sim
3,A questão apresenta uma situação hipotética em...,"[Art. 1.860. Além dos incapazes, não podem tes...","[Art. 1.860, Art. 1.799, Art. 1.801, Art. 1.802]",[A incapacidade relativa de Mariana no momento...,,A incapacidade relativa de Mariana no momento ...,C,Sim
4,A questão apresenta uma situação hipotética em...,[§ 1 o A união estável não se constituirá se o...,[Art. 1.723],[Este artigo estabelece que a união estável nã...,,"Sim, é possível configurar uma união estável e...",C,Sim
5,A questão discute a responsabilidade dos herde...,[nenhum destes será obrigado a pagar senão a q...,[Art. 276],[Este artigo estabelece que os herdeiros não s...,,E,Errado,Sim
6,A questão apresenta um caso de divórcio culpos...,[Art. 1.601. Cabe ao marido o direito de conte...,"[Art. 1.601, Art. 1.639, Art. 1.640]",[O art. 1.601 é relevante pois trata da contes...,,E,E,Sim
7,A sentença afirma que se um dos credores solid...,[Art. 270. Se um dos credores solidários falec...,[Art. 270],[O art. 270 estabelece que cada herdeiro só te...,,Errado,E,Sim
8,"A questão apresenta uma situação em que Diana,...",[Art. 1.813. Quando o herdeiro prejudicar os s...,[Art. 1.813],[Este artigo permite que os credores aceitem a...,,O Banco XYZ pode aceitar a herança e receber u...,D,Sim
9,Análise da questão sobre o Direito das Sucessões.,[Art. 1.858. O testamento é ato personalíssimo...,"[Art. 1.858, Art. 1.857, Art. 1.860]",[O artigo 1.858 confirma que o testamento é at...,,"B) O testamento é ato personalíssimo, podendo ...",B,Sim
